# Module 2, Lesson 2: SQL Fundamentals and Data Extraction

## Setup

In [ ]:
import pandas as pd
import numpy as np
import sqlite3
from datetime import datetime, timedelta

# Display settings for better output
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', 50)

print("Libraries loaded successfully!")

---

## Part 1: Creating Sample Database
**Setting Up Our SQL Environment**

### Creating Tables for Our Examples
We'll create a small retail database with customers, products, and orders. This simulates a real business database you might encounter.

In [ ]:
# Create an in-memory SQLite database
conn = sqlite3.connect(':memory:')
cursor = conn.cursor()

# Create customers table
cursor.execute('''
CREATE TABLE customers (
    customer_id INTEGER PRIMARY KEY,
    name TEXT NOT NULL,
    email TEXT,
    city TEXT,
    state TEXT,
    join_date DATE
)
''')

# Create products table
cursor.execute('''
CREATE TABLE products (
    product_id INTEGER PRIMARY KEY,
    product_name TEXT NOT NULL,
    category TEXT,
    price DECIMAL(10,2)
)
''')

# Create orders table
cursor.execute('''
CREATE TABLE orders (
    order_id INTEGER PRIMARY KEY,
    customer_id INTEGER,
    product_id INTEGER,
    quantity INTEGER,
    order_date DATE,
    FOREIGN KEY (customer_id) REFERENCES customers(customer_id),
    FOREIGN KEY (product_id) REFERENCES products(product_id)
)
''')

print("Database tables created successfully!")

### Populating Tables with Sample Data
Adding realistic data that we'll query throughout the lesson. This represents typical business data you'd analyze.

In [ ]:
# Insert customers
customers_data = [
    (1, 'Alice Johnson', 'alice@email.com', 'New York', 'NY', '2023-01-15'),
    (2, 'Bob Smith', 'bob@email.com', 'Los Angeles', 'CA', '2023-02-20'),
    (3, 'Charlie Brown', 'charlie@email.com', 'Chicago', 'IL', '2023-01-10'),
    (4, 'Diana Prince', 'diana@email.com', 'Houston', 'TX', '2023-03-05'),
    (5, 'Edward Norton', 'edward@email.com', 'Phoenix', 'AZ', '2023-04-12'),
    (6, 'Fiona Green', 'fiona@email.com', 'New York', 'NY', '2023-02-28'),
    (7, 'George Wilson', 'george@email.com', 'Los Angeles', 'CA', '2023-05-01'),
    (8, 'Helen Troy', 'helen@email.com', 'Chicago', 'IL', '2023-03-15')
]

cursor.executemany('INSERT INTO customers VALUES (?,?,?,?,?,?)', customers_data)

# Insert products
products_data = [
    (101, 'Laptop Pro', 'Electronics', 1299.99),
    (102, 'Wireless Mouse', 'Electronics', 29.99),
    (103, 'Office Chair', 'Furniture', 249.99),
    (104, 'Desk Lamp', 'Furniture', 45.99),
    (105, 'Notebook Set', 'Stationery', 15.99),
    (106, 'Coffee Maker', 'Appliances', 89.99),
    (107, 'Water Bottle', 'Accessories', 19.99),
    (108, 'Phone Stand', 'Electronics', 12.99)
]

cursor.executemany('INSERT INTO products VALUES (?,?,?,?)', products_data)

# Insert orders
orders_data = [
    (1001, 1, 101, 1, '2024-01-05'),
    (1002, 2, 102, 2, '2024-01-06'),
    (1003, 1, 105, 3, '2024-01-07'),
    (1004, 3, 103, 1, '2024-01-08'),
    (1005, 4, 106, 1, '2024-01-09'),
    (1006, 2, 101, 1, '2024-01-10'),
    (1007, 5, 107, 4, '2024-01-11'),
    (1008, 6, 102, 1, '2024-01-12'),
    (1009, 3, 108, 2, '2024-01-13'),
    (1010, 7, 104, 2, '2024-01-14'),
    (1011, 1, 106, 1, '2024-01-15'),
    (1012, 4, 102, 3, '2024-01-16'),
    (1013, 8, 105, 5, '2024-01-17'),
    (1014, 5, 103, 1, '2024-01-18'),
    (1015, 2, 107, 2, '2024-01-19')
]

cursor.executemany('INSERT INTO orders VALUES (?,?,?,?,?)', orders_data)
conn.commit()

print("Sample data inserted successfully!")
print(f"- {len(customers_data)} customers")
print(f"- {len(products_data)} products")
print(f"- {len(orders_data)} orders")

---

## Part 2: Basic SELECT Statements
**The Foundation of SQL Queries**

### Example 1: SELECT All Columns
The asterisk (*) means "all columns". This is the simplest query - just show me everything in the table.

In [ ]:
# SQL: SELECT * FROM customers
query = "SELECT * FROM customers"
result = pd.read_sql_query(query, conn)

print("SQL Query:", query)
print("\nResult:")
print(result)

### Example 2: SELECT Specific Columns
Usually you don't need all columns. Selecting only what you need makes queries faster and results cleaner.

In [ ]:
# SQL: SELECT specific columns
query = "SELECT name, city, state FROM customers"
result = pd.read_sql_query(query, conn)

print("SQL Query:", query)
print("\nResult:")
print(result)

### Example 3: Using Column Aliases
Aliases let you rename columns in your output. This is helpful for making results more readable or preparing data for reports.

In [ ]:
# SQL: Using aliases to rename columns
query = """
SELECT 
    product_name AS Product,
    category AS Type,
    price AS 'Price ($)'
FROM products
"""
result = pd.read_sql_query(query, conn)

print("SQL Query:", query)
print("\nResult:")
print(result)

---

## Part 3: Filtering with WHERE
**Finding Specific Records**

### Example 4: Simple WHERE Clause
WHERE filters rows based on conditions. This is like applying a filter in Excel - only show me rows that match my criteria.

In [ ]:
# SQL: Filter with WHERE
query = "SELECT * FROM products WHERE category = 'Electronics'"
result = pd.read_sql_query(query, conn)

print("SQL Query:", query)
print("\nElectronics products only:")
print(result)

### Example 5: Numeric Comparisons
WHERE works with numeric comparisons too. You can use >, <, >=, <=, and != for filtering numbers.

In [ ]:
# SQL: Numeric comparison
query = "SELECT product_name, price FROM products WHERE price < 50"
result = pd.read_sql_query(query, conn)

print("SQL Query:", query)
print("\nProducts under $50:")
print(result)

### Example 6: Multiple Conditions with AND/OR
Combine conditions with AND (both must be true) or OR (either can be true). Parentheses help control the logic.

In [ ]:
# SQL: Multiple conditions with AND
query = """
SELECT * FROM products 
WHERE category = 'Electronics' 
  AND price < 100
"""
result = pd.read_sql_query(query, conn)

print("SQL Query:", query)
print("Affordable electronics (under $100):")
print(result)

print("\n" + "="*50 + "\n")

# SQL: Multiple conditions with OR
query = """
SELECT * FROM customers 
WHERE city = 'New York' 
   OR city = 'Los Angeles'
"""
result = pd.read_sql_query(query, conn)

print("SQL Query:", query)
print("\nCustomers from NY or LA:")
print(result)

### Example 7: Pattern Matching with LIKE
LIKE allows pattern matching in text. The % symbol means "any characters" and _ means "one character".

In [ ]:
# SQL: Pattern matching with LIKE
query = "SELECT * FROM customers WHERE email LIKE '%@email.com'"
result = pd.read_sql_query(query, conn)

print("SQL Query:", query)
print("\nCustomers with @email.com addresses:")
print(result[['name', 'email']])

print("\n" + "="*50 + "\n")

# Another LIKE example
query = "SELECT * FROM products WHERE product_name LIKE '%Chair%' OR product_name LIKE '%Desk%'"
result = pd.read_sql_query(query, conn)

print("SQL Query:", query)
print("\nFurniture items containing 'Chair' or 'Desk':")
print(result)

---

## Part 4: Sorting and Limiting Results
**Organizing Your Query Results**

### Example 8: ORDER BY for Sorting
ORDER BY sorts your results. Default is ascending (ASC), but you can specify descending (DESC) for reverse order.

In [ ]:
# SQL: Sort by price (ascending)
query = "SELECT product_name, price FROM products ORDER BY price"
result = pd.read_sql_query(query, conn)

print("SQL Query:", query)
print("\nProducts from cheapest to most expensive:")
print(result)

print("\n" + "="*50 + "\n")

# SQL: Sort by price (descending)
query = "SELECT product_name, price FROM products ORDER BY price DESC"
result = pd.read_sql_query(query, conn)

print("SQL Query:", query)
print("\nProducts from most expensive to cheapest:")
print(result)

### Example 9: LIMIT to Control Result Size
LIMIT restricts how many rows are returned. Perfect for getting top N results or sampling data.

In [ ]:
# SQL: Get top 3 most expensive products
query = """
SELECT product_name, price 
FROM products 
ORDER BY price DESC 
LIMIT 3
"""
result = pd.read_sql_query(query, conn)

print("SQL Query:", query)
print("\nTop 3 most expensive products:")
print(result)

---

## Part 5: Aggregation with GROUP BY
**Summarizing Data**

### Example 10: Basic Aggregation Functions
SQL has built-in functions for calculations: COUNT, SUM, AVG, MIN, MAX. These work on entire tables or groups of rows.

In [ ]:
# SQL: Count total customers
query = "SELECT COUNT(*) as total_customers FROM customers"
result = pd.read_sql_query(query, conn)
print("SQL Query:", query)
print("Result:", result.iloc[0,0], "customers")

print("\n" + "="*50 + "\n")

# SQL: Statistics on product prices
query = """
SELECT 
    COUNT(*) as product_count,
    MIN(price) as cheapest,
    MAX(price) as most_expensive,
    AVG(price) as average_price,
    SUM(price) as total_value
FROM products
"""
result = pd.read_sql_query(query, conn)

print("SQL Query:", query)
print("\nProduct Statistics:")
for col in result.columns:
    print(f"{col}: ${result[col][0]:.2f}")

### Example 11: GROUP BY for Categories
GROUP BY creates groups of rows that share values. Then you can apply aggregate functions to each group separately.

In [ ]:
# SQL: Group by category
query = """
SELECT 
    category,
    COUNT(*) as product_count,
    AVG(price) as avg_price
FROM products
GROUP BY category
ORDER BY avg_price DESC
"""
result = pd.read_sql_query(query, conn)

print("SQL Query:", query)
print("\nProducts by category:")
print(result)

### Example 12: GROUP BY with Multiple Columns
You can group by multiple columns to create more detailed breakdowns. This creates a hierarchy of groups.

In [ ]:
# SQL: Group by city and state
query = """
SELECT 
    state,
    city,
    COUNT(*) as customer_count
FROM customers
GROUP BY state, city
ORDER BY customer_count DESC
"""
result = pd.read_sql_query(query, conn)

print("SQL Query:", query)
print("\nCustomers by location:")
print(result)

### Example 13: HAVING to Filter Groups
HAVING is like WHERE but for groups. Use it to filter results after grouping, not individual rows.

In [ ]:
# SQL: Find categories with average price > $50
query = """
SELECT 
    category,
    COUNT(*) as product_count,
    AVG(price) as avg_price
FROM products
GROUP BY category
HAVING AVG(price) > 50
"""
result = pd.read_sql_query(query, conn)

print("SQL Query:", query)
print("\nCategories with average price over $50:")
print(result)

---

## Part 6: Joining Tables
**Combining Data from Multiple Tables**

### Example 14: INNER JOIN
INNER JOIN combines rows from two tables where the join condition matches. This is the most common type of join.

In [ ]:
# SQL: Join orders with customer names
query = """
SELECT 
    o.order_id,
    c.name as customer_name,
    o.order_date,
    o.quantity
FROM orders o
INNER JOIN customers c ON o.customer_id = c.customer_id
ORDER BY o.order_date
LIMIT 10
"""
result = pd.read_sql_query(query, conn)

print("SQL Query:", query)
print("\nOrders with customer names:")
print(result)

### Example 15: Multiple JOINs
You can join multiple tables together to build complete pictures of your data. This combines customers, orders, and products.

In [ ]:
# SQL: Join all three tables
query = """
SELECT 
    o.order_id,
    c.name as customer,
    p.product_name as product,
    o.quantity,
    p.price,
    (o.quantity * p.price) as total_amount
FROM orders o
INNER JOIN customers c ON o.customer_id = c.customer_id
INNER JOIN products p ON o.product_id = p.product_id
ORDER BY total_amount DESC
LIMIT 10
"""
result = pd.read_sql_query(query, conn)

print("SQL Query:", query)
print("\nComplete order details with calculated totals:")
print(result)

### Example 16: LEFT JOIN
LEFT JOIN keeps all rows from the left table, even if there's no match in the right table. Useful for finding missing relationships.

In [ ]:
# First, let's add a customer with no orders
cursor.execute("INSERT INTO customers VALUES (9, 'Zara Young', 'zara@email.com', 'Boston', 'MA', '2024-01-01')")
conn.commit()

# SQL: Find customers and their order counts (including those with no orders)
query = """
SELECT 
    c.name,
    COUNT(o.order_id) as order_count
FROM customers c
LEFT JOIN orders o ON c.customer_id = o.customer_id
GROUP BY c.customer_id, c.name
ORDER BY order_count DESC
"""
result = pd.read_sql_query(query, conn)

print("SQL Query:", query)
print("\nAll customers with their order counts:")
print(result)
print("\nNotice: Zara Young has 0 orders (LEFT JOIN kept her in results)")

---

## Part 7: Complex Queries
**Putting It All Together**

### Example 17: Business Intelligence Query
Real-world queries combine multiple concepts. This query finds our best customers by total spending.

In [ ]:
# SQL: Find top customers by total spending
query = """
SELECT 
    c.name as customer_name,
    c.city,
    c.state,
    COUNT(DISTINCT o.order_id) as total_orders,
    SUM(o.quantity) as total_items,
    SUM(o.quantity * p.price) as total_spent
FROM customers c
INNER JOIN orders o ON c.customer_id = o.customer_id
INNER JOIN products p ON o.product_id = p.product_id
GROUP BY c.customer_id, c.name, c.city, c.state
HAVING total_spent > 100
ORDER BY total_spent DESC
"""
result = pd.read_sql_query(query, conn)

print("SQL Query:", query)
print("\nTop customers by spending:")
print(result)

### Example 18: Category Performance Analysis
This query analyzes which product categories are performing best, combining multiple tables and calculations.

In [ ]:
# SQL: Category performance analysis
query = """
SELECT 
    p.category,
    COUNT(DISTINCT o.order_id) as number_of_orders,
    SUM(o.quantity) as units_sold,
    SUM(o.quantity * p.price) as revenue,
    AVG(o.quantity * p.price) as avg_order_value
FROM products p
INNER JOIN orders o ON p.product_id = o.product_id
GROUP BY p.category
ORDER BY revenue DESC
"""
result = pd.read_sql_query(query, conn)

print("SQL Query:", query)
print("\nCategory Performance Report:")
print(result)

### Example 19: SQL vs Pandas Comparison
Let's see how the same analysis looks in both SQL and pandas. This helps you understand both approaches.

In [ ]:
# First, load all data into pandas DataFrames
customers_df = pd.read_sql_query("SELECT * FROM customers", conn)
products_df = pd.read_sql_query("SELECT * FROM products", conn)
orders_df = pd.read_sql_query("SELECT * FROM orders", conn)

print("=" * 60)
print("SAME ANALYSIS: SQL vs PANDAS")
print("=" * 60)

# SQL approach
print("\nSQL APPROACH:")
sql_query = """
SELECT category, AVG(price) as avg_price 
FROM products 
GROUP BY category
"""
sql_result = pd.read_sql_query(sql_query, conn)
print(sql_result)

# Pandas approach
print("\nPANDAS APPROACH:")
pandas_result = products_df.groupby('category')['price'].mean().reset_index()
pandas_result.columns = ['category', 'avg_price']
print(pandas_result)

print("\nBoth give the same result!")

### Cleanup
Close the database connection when done.

In [ ]:
# Close the connection
conn.close()
print("Database connection closed.")